In [123]:
#mohana palaka
#9.9.2020
#spellcheck, as decribed by peter norvig in https://norvig.com/spell-correct.html

In [95]:
import re
import chardet #to detect the encoding of the file; it may not always be utf-8
from collections import Counter

In [96]:
#function to derive a library of words to form the basis of our spellchecker
def get_library(filename):
    
    words = []
    
    file = open(filename, 'rb').read()
    
    file_info = chardet.detect(file)
    encoding = file_info['encoding']
        
    file = open(filename, encoding = encoding)
    doc = file.read().replace("\n", " ")
    file.close()
    
    lower_doc = doc.lower()
    
    words = re.findall(r'\w+',lower_doc)
    lib = set(words)
    
    return words,lib

#get the probabilities of all the words in the vocabulary
def get_probs(raw_words):
    
    counts = Counter(raw_words) #dict to store the counts of each word
    all_words = sum(counts.values())
    
    probs = {} #dict to store probabilities
    
    for word in counts:
        probs[word] = counts[word]/all_words #basic probability; use bayes?
    
    return probs

#functions for all the possible operations/edits
def letter_del(word):
    
    split = [(word[:i], word[i:]) for i in range(len(word))]
    
    deletes = [L + R[1:] for L,R in split if R]
    
    return deletes


def letter_ins(word):
    
    alphabet='abcdefghijklmnopqrstuvwxyz'
    
    split = [(word[:i], word[i:]) for i in range(len(word)+1)]
    
    inserts = [L + alphabet[i] + R for L,R in split for i in range(len(alphabet))]
    
    return inserts

def letter_rep(word):
    
    alphabet='abcdefghijklmnopqrstuvwxyz'
    
    split = [(word[:i], word[i:]) for i in range(len(word))]
    
    rep = [L + alphabet[i] + R[1:] for L,R in split for i in range(len(alphabet))]
    
    #removing duplicates and the word itself
    rep_set = set(rep)
    rep_set.discard(word)
    
    rep = list(rep_set)
    
    return rep

#combining all the different types of edits of the mispelled word
def one_letter_edits(word):
   
    all_edits = set(letter_del(word) + letter_ins(word) + letter_rep(word))
    
    return all_edits
    
    
def two_letter_edits(word):
    
    #editing the list of one letter edited word will give us all two letter edited words
    one_letter_edits_l = one_letter_edits(word)
    
    all_edits = set(letter_del(word) + letter_ins(word) + letter_rep(word))

    return all_edits

def autocorrect(word, probs):
    
    #keep the one and two letter edits seperate; we want to prioritize the one letter edited words
    all_cands_one = one_letter_edits(word) 
    all_cands_two = two_letter_edits(word)  
        
    #keep a word only if it is a word in the vocabulary
    viable_one = [word for word in all_cands_one if word in probs.keys()]
    viable_two = [word for word in all_cands_two if word in probs.keys()]

        
    viable_probs_one, viable_probs_two = {},{}
    
    for cand in viable_one: 
        viable_probs_one[cand] = probs.get(cand,0)
    for cand2 in viable_two:
        viable_probs_two[cand2] = probs.get(cand2,0)

    correct_word_one = Counter.most_common(viable_probs_one, 2)
    correct_word_two = Counter.most_common(viable_probs_two, 2)
    
    correct_word = correct_word_one or correct_word_two #preference is given to a one letter edit word; 
                                                        #only if the list is empty, it suggests a two letter edit word 
    
    best_word = [tup_word for tup_word, tup_prob in correct_word]

    return f'Hmm... {word}? Did you mean {best_word[0]} or {best_word[1]}?'
    

In [112]:
txtfile = 'war-and-peace.txt' #i used the book war and peace as my text corpus
words, lib = get_library(txtfile) #the word 'hello' doesnt appear even once in war and peace :0

print(f'there are {len(lib)} unique words in {txtfile}')

there are 17736 unique words in war-and-peace.txt


In [116]:
#get the dictionary of probabilities   
probs = get_probs(words)

In [121]:
#change this to any mispelled word 
mispelled = 'sloe'

In [122]:
autocorrect(mispelled, probs)

'Hmm... sloe? Did you mean slope or slow?'